~~to run
/Users/julianprokofiev/opt/anaconda3/envs/my_env/bin/python~~

~~source /Users/julianprokofiev/opt/anaconda3/envs/my_env/bin/activate
your-venv/bin/activate~~

<a href="https://colab.research.google.com/github/goghino/deep-opf/blob/master/data/jupyter/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Import the data</h1>

For case30: 20 loads (input), 30 LAM_P's (output)
<ul>
    <li> Reduce dimensionality by removing unnecessary inputs (columns of 0's)
    <li> After getting Tensorflow running, try regressing only to one output (reference bus, refer to Juraj's email)
    <li> Depending on how small the changes in LAM_P are, may need different numerical representation of results
</ul>


For reference:
<p float="left">
    <img src="matpower_glossary_1.png" height="400" />
    <img src="matpower_glossary_2.png" height="400" />
    <img src="matpower_glossary_3.png" height="400" />
</p> 

In [114]:
import csv
import numpy as np
import pandas as pd


if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    datapath = 'https://raw.githubusercontent.com/goghino/deep-opf/master/data/'
    path_PD = datapath + 'csv/sample_case30_PD.csv'
    path_PG = datapath + 'csv/sample_case30_PG.csv'
    path_LAM_P = datapath + 'csv/sample_case30_LAM_P.csv'
else:
    print('Not running on CoLab')
    path_PD = '../csv/sample_case30_PD.csv'
    path_PG = '../csv/sample_case30_PG.csv'
    path_LAM_P = '../csv/sample_case30_LAM_P.csv'

PD = pd.read_csv(path_PD, delimiter=',', header=None)
PG = pd.read_csv(path_PG, delimiter=',', header=None)
LAM_P = pd.read_csv(path_LAM_P, delimiter=',', header=None)

print(path_PD)

print(PD.shape)
print(LAM_P.shape)

Running on CoLab
https://raw.githubusercontent.com/goghino/deep-opf/master/data/csv/sample_case30_PD.csv
(3111, 30)
(3111, 30)


Remove excess columns in PD
https://stackoverflow.com/questions/21164910/how-do-i-delete-a-column-that-contains-only-zeros-in-pandas

*then* convert to numpy format

In [115]:
PD = PD.loc[:, (PD != 0).any(axis=0)]
print(PD.shape)

# convert to numpy format
PD = PD.to_numpy(dtype=np.float32)
PG = PG.to_numpy(dtype=np.float32)
LAM_P = LAM_P.to_numpy(dtype=np.float32)

(3111, 20)


In [116]:
from sklearn.model_selection import train_test_split

# the parameter we'll be regressing onto. Select:
# - "LAM_P" for Lagrange multiplier on real power mismatch (for the buses)
# - "PG" for real power output (for the generators)
toggle = 'LAM_P'
single_param = 1

if toggle == 'LAM_P':
  # organize the training data: split & shuffle
  PD_train, PD_test, LAM_P_train, LAM_P_test = \
      train_test_split(PD, LAM_P, test_size=0.2, random_state=42, shuffle=True)

  # split test into test & validation
  PD_valid, PD_test, LAM_P_valid, LAM_P_test = \
      train_test_split(PD_test, LAM_P_test, test_size=0.5, random_state=42, shuffle=True)

  print('PD_train shape: ', PD_train.shape)
  print('PD_valid shape: ', PD_valid.shape)
  print('PD_test shape: ', PD_test.shape)
  print('LAM_P_train shape: ', LAM_P_train.shape)
  print('LAM_P_valid shape: ', LAM_P_valid.shape)
  print('LAM_P_test shape: ',LAM_P_test.shape)
  # print(id(PD))
  # print(id(PD_train))
  # print(id(PD_copy))

  
elif toggle == 'PG':
    # organize the training data: split & shuffle
  PD_train, PD_test, PG_train, PG_test = \
      train_test_split(PD, PG, test_size=0.2, random_state=42, shuffle=True)

  # split test into test & validation
  PD_valid, PD_test, PG_valid, PG_test = \
      train_test_split(PG_test, PG_test, test_size=0.5, random_state=42, shuffle=True)

  print('PD_train shape: ', PD_train.shape)
  print('PD_valid shape: ', PD_valid.shape)
  print('PD_test shape: ', PD_test.shape)
  print('PG_train shape: ', PG_train.shape)
  print('PG_valid shape: ', PG_valid.shape)
  print('PG_test shape: ', PG_test.shape)
  # print(id(PD))
  # print(id(PD_train))
  # print(id(PD_copy))

################################################################################

print('LAM_P_train shape: ', LAM_P_train.shape)
# LAM_P_train = LAM_P_train[:,0]
print('LAM_P_train shape: ', LAM_P_train.shape)



PD_train shape:  (2488, 20)
PD_valid shape:  (311, 20)
PD_test shape:  (312, 20)
LAM_P_train shape:  (2488, 30)
LAM_P_valid shape:  (311, 30)
LAM_P_test shape:  (312, 30)
LAM_P_train shape:  (2488, 30)
LAM_P_train shape:  (2488, 30)


~~Copy & save the data before normalization process~~

Unnecessary, can be done bug-free using fit_transform()

In [117]:
#@title old bad standardizing
# https://stackoverflow.com/questions/34192290/ipython-notebook-elegant-way-of-turning-off-part-of-cells
%%script false --no-raise-error

# # create copy before normalizing
PD_copy = np.vstack((PD_train, PD_valid, PD_test))
print(PD_copy.shape)
print(id(PD))
print(id(PD_copy))

if toggle == 'LAM_P':
  LAM_P_copy = np.vstack((LAM_P_train, LAM_P_valid, LAM_P_test))
  # LAM_P_copy = copy.deepcopy(LAM_P_train + LAM_P_valid + LAM_P_test)
  print('LAM_P_copy shape: ', LAM_P_copy.shape)
  print('LAM_P_train shape: ', LAM_P_train.shape)
  # print(id(LAM_P))
  # print(id(LAM_P_copy))


  # print(LAM_P_copy.shape)
  # print(LAM_P_copy.dtype)
  # print(LAM_P_copy) ??? why random numbers on the side

  print(LAM_P_train[1][1])

  # LAM_P_train[1][1] = 5
  # print(LAM_P_train[1][1])

  print(LAM_P_copy[1][1])


# # normalize
PD_train -= np.mean(PD_train, axis=0)
PD_train /= np.std(PD_train, axis=0)
PD_valid -= np.mean(PD_valid, axis=0)
PD_valid /= np.std(PD_valid, axis=0)
PD_test -= np.mean(PD_test, axis=0)
PD_test /= np.std(PD_test, axis=0)

LAM_P_train -= np.mean(LAM_P_train, axis=0)
LAM_P_train /= np.std(LAM_P_train, axis=0)
LAM_P_valid -= np.mean(LAM_P_valid, axis=0)
LAM_P_valid /= np.std(LAM_P_valid, axis=0)
LAM_P_test -= np.mean(LAM_P_test, axis=0)
LAM_P_test /= np.std(LAM_P_test, axis=0)

### new good standardizing

In [118]:
# now using the proper functions
from sklearn.preprocessing import StandardScaler

if single_param:
  LAM_P_train = LAM_P_train[:,0]
  LAM_P_train = LAM_P_train.reshape(-1,1)
  LAM_P_valid = LAM_P_valid[:,0]
  LAM_P_valid = LAM_P_valid.reshape(-1,1)
  LAM_P_test = LAM_P_test[:,0].reshape(-1,1)
  LAM_P_test = LAM_P_test.reshape(-1,1)

PD_scaler = StandardScaler()
PD_train = PD_scaler.fit_transform(PD_train)
PD_valid = PD_scaler.transform(PD_valid)      # uses same mean and std as PD_train fit


# # for debugging
# print(PD_train[0, :])
# PD_scaler.fit(PD_train)
# PD_scaler.mean_                                    # prints mean of columns
# PD_scaler.scale_                                   # prints stddev of columns
# LAM_P_scaler.mean_
# LAM_P_scaler.scale_

if toggle == 'LAM_P': 
  LAM_P_scaler = StandardScaler()
  LAM_P_train = LAM_P_scaler.fit_transform(LAM_P_train)
  LAM_P_valid = LAM_P_scaler.transform(LAM_P_valid)

elif toggle == 'PG':
  PG_scaler = StandardScaler()
  PG_train = PG_scaler.fit_transform(PG_train)
  PG_valid = PG_scaler.transform(PG_valid)

print('LAM_P_train shape: ', LAM_P_train.shape)
print('LAM_P_valid shape: ', LAM_P_valid.shape)
print('LAM_P_test shape: ',LAM_P_test.shape)



LAM_P_train shape:  (2488, 1)
LAM_P_valid shape:  (311, 1)
LAM_P_test shape:  (312, 1)


Training NN with PD as inputs and PG as outputs

<b>NOTE:</b> The loss function in the paper combines MSE with a penalty term for infeasibility. Include feasibility later

In [119]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf


## Prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

## Train the network

# For regressing onto the reference bus 
if toggle == 'LAM_P':
  # only using first column
  if single_param:
    nn_model = Sequential()
    nn_model.add(Dense(16, activation='tanh'))
    nn_model.add(Dense(16, activation='tanh'))
    nn_model.add(Dense(1, activation='linear'))

    nn_model.compile(optimizer='adam',
                    loss='mse')
    nn_model.fit(PD_train, LAM_P_train, epochs=3000, validation_data=(PD_valid, LAM_P_valid), verbose=1, callbacks = [es])

  # For regressing onto all 30 nodes
  else:
    nn_model = Sequential()
    nn_model.add(Dense(16, activation='tanh'))
    nn_model.add(Dense(16, activation='tanh'))
    nn_model.add(Dense(30, activation='linear'))

    nn_model.compile(optimizer='adam',
                    loss='mse')

    nn_model.fit(PD_train, LAM_P_train, epochs=3000, validation_data=(PD_valid, LAM_P_valid), verbose=1, callbacks = [es])

################################################################################

from sklearn.metrics import mean_squared_error

# not inverse transformed!
train_mse = nn_model.evaluate(PD_train, LAM_P_train, verbose=0)
valid_mse = nn_model.evaluate(PD_valid, LAM_P_valid, verbose=0)

# should I transform() input, then inverse_transform() output?
test_mse = nn_model.evaluate(PD_test, LAM_P_test, verbose=0)

print('Train: ', train_mse)
print('Valid: ', valid_mse) 
print('Test (fake): ', test_mse)

# nn_test_predictions = nn_model.predict(PD_test)
# nn_mse = mean_squared_error(LAM_P_test, nn_test_predictions)



Epoch 1/3000
78/78 [==============================] - 1s 4ms/step - loss: 0.2165 - val_loss: 0.0196
Epoch 2/3000
78/78 [==============================] - 0s 3ms/step - loss: 0.1585 - val_loss: 0.0163
Epoch 3/3000
78/78 [==============================] - 0s 2ms/step - loss: 0.1483 - val_loss: 0.0199
Epoch 4/3000
78/78 [==============================] - 0s 2ms/step - loss: 0.1421 - val_loss: 0.0137
Epoch 5/3000
78/78 [==============================] - 0s 2ms/step - loss: 0.1346 - val_loss: 0.0140
Epoch 6/3000
78/78 [==============================] - 0s 2ms/step - loss: 0.1286 - val_loss: 0.0141
Epoch 7/3000
78/78 [==============================] - 0s 3ms/step - loss: 0.1245 - val_loss: 0.0148
Epoch 8/3000
78/78 [==============================] - 0s 2ms/step - loss: 0.1200 - val_loss: 0.0139
Epoch 9/3000
78/78 [==============================] - 0s 2ms/step - loss: 0.1161 - val_loss: 0.0140
Epoch 10/3000
78/78 [==============================] - 0s 2ms/step - loss: 0.1112 - val_loss: 0.0141

In [122]:
# not inverse transformed!
train_mse = nn_model.evaluate(PD_train, LAM_P_train, verbose=0)
valid_mse = nn_model.evaluate(PD_valid, LAM_P_valid, verbose=0)

# should I transform() input, then inverse_transform() output?
# test_mse = nn_model.evaluate(PD_test, LAM_P_test, verbose=0)

# https://stackoverflow.com/questions/48973140/how-to-interpret-mse-in-keras-regressor
PD_test_scaled = PD_scaler.transform(PD_test)
test_predictions = nn_model.predict(PD_test_scaled)
test_mse = mean_squared_error(LAM_P_test, LAM_P_scaler.inverse_transform(test_predictions))

print('Train: ', train_mse)
print('Valid: ', valid_mse) 

# print(LAM_P_test.shape)
# print('Test predictions shape: ', test_predictions.shape)     # why is it 312 parameters? # of points in test set
print('Test (real): ', test_mse)

# nn_test_predictions = nn_model.predict(PD_test)
# nn_mse = mean_squared_error(LAM_P_test, nn_test_predictions)

Train:  0.0007969780126586556
Valid:  0.004754812456667423
Test (real):  0.0023164132
